In [ ]:
! python -m pip install gurobipy
! python -m pip install pandas

In [ ]:
import json
import gurobipy as gp
from gurobipy import GRB
import pandas as pd


# LETTURA DEI DATI
df = pd.read_csv("data/telecomandi.csv")
# imposto la colonna i come indice di riga per poter accedere ad una riga del file utilizzando .loc
df = df.set_index('risorsa')


# Definizione degli insiemi
## Insieme I dei beni  che possono essere prodotti
"""
Ottengo l’insieme I (beni da produrre):
lista dei nomi delle colonne del dataframe, esclusa la colonna 'quantita'
I = ['telecomando_A', 'telecomando_B']
"""
I = df.drop(columns="quantita").columns.tolist()

## Insieme J delle risorse disponibli
"""
Ottengo l’insieme J (risorse disponibili):
lista dei nomi delle righe del dataframe, esclusa la riga 'profitto'
J = ['display', 'logica', 'trasmettitori', 'tastierini', 'navigazione', 'led']
"""
J = df.drop(index="profitto").index.tolist()

# Definizione dei parametri
## Parametri P: profitto unitario per il bene i in I
"""
seleziono la riga con indice 'profitto' e la converto in dizionario le cui chiavi saranno i valori delle colonne
P = { 'telecomando_A' : 3.0,
      'telecomando_B' : 8.0 }
"""
P = df.loc["profitto"].to_dict()

## Parametri Q: quantità disponibile della risorsa j in J
"""
seleziono la colonna 'quantita' e la converto in dizionario le cui chiavi saranno gli indici delle righe

Q = { "display": 10,
      "logica": 18,
      "trasmettitori": 12,
      "tastierini": 21,
      "navigazione": 9,
      "led": 10 }
"""
Q = df["quantita"].to_dict()

## Parametri A (matrice): quantità di risorsa j necessaria per la produzione di un'unità di del bene i
"""
rimuovo la riga 'profitto' e la colonna 'quantita' per ottenere la matrice A
A[j,i] è la quantita di risorsa j necessaria per la produzione di un'unità di bene i
NB: gli indici sono A_ji e non A_ij per come è strutturato il file csv
"""
A = df.drop(index="profitto").drop(columns="quantita")



print("Beni (I):", I)
print("Risorse (J):", J)
print("Profitti (P):", P)
print("Quantità (D):", Q)
print("Consumi unitari (A):\n", A)


# Creazione modello
m = gp.Model("mix_opt_telecomandi")
m.setParam("OutputFlag", 0)

# Creazione variabili decisionali
x = {}
for i in I:
  x[f"{i}"] = m.addVar(vtype=GRB.INTEGER, lb=0, name=f"x_{i}")
  # x[i] è equivalente

# equivalente anche la seguente riga di codice
# x = m.addVars(I, vtype=GRB.INTEGER, lb=0, name="x")

# Aggiunta vincoli
for j in J:
    sum = gp.quicksum(A.loc[j, i] * x[i] for i in I)
    m.addConstr(
        sum <= Q[j],
        name=f"vincolo_{j}",
    )

# O.f.
m.setObjective(gp.quicksum(P[i] * x[i] for i in I), GRB.MAXIMIZE)

m.optimize()

# Visualizzazione risultato
status_names = {
    GRB.OPTIMAL: "OPTIMAL",
    GRB.SUBOPTIMAL: "SUBOPTIMAL",
    GRB.INFEASIBLE: "INFEASIBLE",
    GRB.UNBOUNDED: "UNBOUNDED",
    GRB.INF_OR_UNBD: "INF_OR_UNBD",
}
print(f"\nModel status: {status_names.get(m.status, m.status)}")

if m.status == GRB.OPTIMAL:
    print(f"Objective (costo) = {m.objVal:.6g}\n")
    for i in I:
        if x[i].X > 1e-6:
            print(f"x[{i}] = {x[i].X:.0f}")